In [106]:
import numpy as np

In [107]:
import sys
sys.path.append(r"C:\_Lib\python")
sys.path.append(r"C:\Users\eeshg\Documents\multimode_sims\Design\fab")

from mask_maker import * 
# from slab.circuits.mp_components import *

In [108]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
from mm_qubit import *
from mm_chips_base import * 

# Imports

# Mask Generation

## Wafer Initialization

 4in wafer -> diameter=101600, flat_distance=48200


 3in wafer -> diameter=76200., flat_distance=37100.

 
 2in wafer -> diameter=50800., flat_distance=24100.

In [110]:
MaskName = 'DravidQubits_opt'
save_folder=r"C:\Users\eeshg\Documents\multimode_sims\Design\fab\LMM4_Dravid\designs_dxf"
# m = WaferMask(MaskName, diameter=101600., flat_angle=270., flat_distance=48200., wafer_padding=2500,
#                   chip_size=(1600, 25950),
#                   dicing_border=400, etchtype=False, wafer_edge=False,
#                   dashed_dicing_border=80, ndashes=3, dice_corner=True, square_arr=False)
m = WaferMask(MaskName, diameter=101600., flat_angle=270., flat_distance=48200., wafer_padding=2500,
                  chip_size=(1600, 25950),
                  dicing_border=200, etchtype=False, wafer_edge=True,
                  dashed_dicing_border=80, ndashes=2, dice_corner=False, square_arr=False)

Maximum number of rows=3 and cols=53
Room for 113 chips on wafer.


## Chips

In [111]:
## Iniatilization
mm_chips_base = MMChipsBase()
chip_defaults = mm_chips_base.set_mask_init()

### Chip Configs

In [112]:
import itertools

# Generate 10 unique (width, gap) pairs
widths = [0.39, 0.22, 0.16, 0.12, 0.1, 0.39, 0.22, 0.16, 0.12, 0.1 ]
gaps = [0.2]*10
names = [chr(ord('A') + i) for i in range(len(widths))]


chip_configs = [
    ({'width': float(w), 'gap': float(g)}, False, names[i])#, 1 if i % 2 == 0 else 0)
    for i, (w, g) in enumerate(zip(widths, gaps))
]

chip_configs

[({'width': 0.39, 'gap': 0.2}, False, 'A'),
 ({'width': 0.22, 'gap': 0.2}, False, 'B'),
 ({'width': 0.16, 'gap': 0.2}, False, 'C'),
 ({'width': 0.12, 'gap': 0.2}, False, 'D'),
 ({'width': 0.1, 'gap': 0.2}, False, 'E'),
 ({'width': 0.39, 'gap': 0.2}, False, 'F'),
 ({'width': 0.22, 'gap': 0.2}, False, 'G'),
 ({'width': 0.16, 'gap': 0.2}, False, 'H'),
 ({'width': 0.12, 'gap': 0.2}, False, 'I'),
 ({'width': 0.1, 'gap': 0.2}, False, 'J')]

In [113]:
names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

## Configs specific to Multimode Qubit Chip

In [114]:
qubit_optical_configs= {
    'x_thin': 30,
    'y_right_thin': 350,
    'x_right_thick': 150,
    'y_right_thick': 3100,
    'y_left_thin': 370,
    'x_left_thick': 150,
    'y_left_thick': 4100,
    'jjgap': 30,
    'xpos': 1000 - 200,
    'ypos': 17700 - 200 + 7780 + 350,
    
}

In [115]:
test_qubit_optical_configs = {
    'x_thin': 30,
    'y_right_thin': 100,
    'x_right_thick': 800,
    'y_right_thick': 800,
    'y_left_thin': 100,
    'x_left_thick': 800,
    'y_left_thick': 800,
    'jjgap': 30,
}
yoffset = 2000
test_qubit_optical_posns = [
    (800, 3000+200), 
    (800, 3000+ yoffset+200),
    (800, 3000+2*yoffset +200),
    (800, 3000+3*yoffset +200),
    #(800, 3000+4*yoffset +200),
]

In [116]:

qubit_junction_configs = dict(
    bar_width=20,
    bar_length=17,
    thin_bar_length = 8, # this is the length for top bar
    # taper_length=5, # this i
    taper_length1=5,  # this is the first taper (bottom bar)
    taper_length2=5,  # this is the second taper (bottom bar) 
    thin_bar_width1=2,  # this is the width of the first thin bar
    thin_bar_length1=2.5,  # this is the length of the first thin bar (bottom bar)
    thin_bar_length2=2.5,  # this is the length of the second thin bar(bottom bar)
    top_bar_taper_length=5,
    jj_length=0.8,
    # jj_width=junc_correction['width'],
    # jj_gap=junc_correction['gap'],
    top_bar_jj_width=2,
    # under cut parameters 
    undercut_width = 0.2,
    undercut_width_pads = 0.7,
    # for horizontal oriented junctions 07/27 blunder
    # horizontal_l_arm = 70+15,
    # bend_radius = 5,
    short = False,
    open = False

)
test_qubit_junction_type=[{'open': True, 'short': False},
                          {'open': False, 'short': True},
                          {'open': False, 'short': False},
                          {'open': False, 'short': False}
                          ]


### Chip Generation

In [117]:
draw_optical = True
# draw_optical = False
draw_ebeam = False
# draw_ebeam = False
two_layer = True if draw_ebeam == 1 else False

In [118]:
from copy import copy
chips = []
author = 'Eesh Gupta'
# flag flips the junction vertically 
for idx, (junc_correction, flip, name) in enumerate(chip_configs):
    chip = Chip(
        name,
        author="",
        size=m.chip_size,
        mask_id_loc=(5800, 6430),
        chip_id_loc=(200, 100),
        textsize=(300, 300),
        two_layer=two_layer,
        solid=False,
    )
    full_chip_drawer = MultimodeQubitChip(structure=chip, chip_defaults=chip_defaults, flip=flip)
    full_chip_drawer.chipInit(chip, defaults=chip_defaults)
    if draw_optical: 
        draw_marks = True if idx == 0 or idx == (len(chip_configs)-1) else False
        full_chip_drawer.draw_alignment_marks(draw_marks=draw_marks)

    if draw_optical:
        full_chip_drawer.draw_multimode_qubit_optical(qubit_optical_configs=qubit_optical_configs,
                                                      flag = not flip,xpos= qubit_optical_configs['xpos'],
                                                      ypos= qubit_optical_configs['ypos'])
        #test structures
        for idx_test, (xpos, ypos) in enumerate(test_qubit_optical_posns):
            full_chip_drawer.draw_multimode_qubit_optical(qubit_optical_configs=test_qubit_optical_configs,
                                                          flag = not flip, 
                                                          xpos = xpos, ypos = ypos)
            chip.label_chip(drawing=chip, maskid=None, chipid=name + str(idx_test),
                     author=chip.author, offset=(xpos - 300, ypos -50), textsize = (100,100))
        # names 
        # CHIP NAME
        chip.label_chip(drawing=chip, maskid=None, chipid=name,
                     author=chip.author, offset=(300, 20))
        
    if draw_ebeam:
        # print(f"Drawing e-beam features for {name} with flip={flip}")
        full_chip_drawer.draw_multimode_qubit_ebeam(junction_configs=qubit_junction_configs,
                                                    junc_correction=junc_correction, flag= not flip, 
                                                    xpos = qubit_optical_configs['xpos'],
                                                    ypos = qubit_optical_configs['ypos'] - qubit_optical_configs['y_left_thick'] - qubit_optical_configs['y_left_thin'] - qubit_optical_configs['jjgap'] )#- qubit_optical_configs['x_thin'])
        ##test structures
        for idx_test, (xpos, ypos) in enumerate(test_qubit_optical_posns):
            test_qubit_junction_configs = copy(qubit_junction_configs)
            test_qubit_junction_configs['short'] = test_qubit_junction_type[idx_test]['short']
            test_qubit_junction_configs['open'] = test_qubit_junction_type[idx_test]['open']
            full_chip_drawer.draw_multimode_qubit_ebeam(junction_configs=test_qubit_junction_configs,
                                                        junc_correction=junc_correction, flag= not flip,
                                                        xpos = xpos, 
                                                        ypos = ypos - test_qubit_optical_configs['y_left_thick'] - test_qubit_optical_configs['y_left_thin'] - test_qubit_optical_configs['jjgap'] )#- test_qubit_optical_configs['x_thin'])

    # draw_bridge_junction(chip, d, junc_correction, flip)
    chips.append(chip)





In [119]:
square

0

## Addings Chips to wafer

In [120]:
# add chips to wafer
chip_fnames = []
for idx, c in enumerate(chips):
    name = m.add_chip(c, 1, label= True, save_folder=save_folder)
    chip_fnames.append(name)
chip_fnames

['C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-A.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-B.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-C.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-D.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-E.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-F.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-G.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-H.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\fab\\LMM4_Dravid\\designs_dxf\\DRAVIDQUBITS_OPT-I.dxf',
 

In [121]:
with open("chip_names_qubit.txt", "w") as f:
    for name in chip_fnames:
        f.write(name + "\n")


# Output Wafer

In [122]:
import os
import subprocess
from time import sleep

def save_and_open_mask(m, folder_path, klayout_path, sleep_time=0.1):

    mask_name = m.name
    mask_file_path = os.path.join(folder_path, mask_name + '.dxf')
    m.save(name=mask_file_path)

    print("\n\n Chip names are:")
    print("_____________________")
    # for name in chip_names:
    #     print(name)
    print("_____________________\n\n")

    sleep(sleep_time)

    def open_klayout_app(klayout_path, mask_file_path):
        subprocess.Popen(f'"{klayout_path}" "{mask_file_path}" -e')

    open_klayout_app(klayout_path, mask_file_path)

# Usage:
folder_path = save_folder
klayout_path = r"C:\Users\eeshg\AppData\Roaming\KLayout\klayout_app"
save_and_open_mask(m, folder_path, klayout_path)





 Chip names are:
_____________________
_____________________




In [123]:
2/2+1

2.0